# DEEPMIND ALPHAZERO CHESS

## TODO
- DONE!! Experiences are not stored properly since ZH is sometimes not avaiable in MCTS storage
- DONE!! DOUBLE CHECK!!! Handle the hidden state properly as the AI is navigating the tree! 


## TODO LATER
#### - Get backward possibly loss to train the controller. Might need to save the tensor output like states, values, etc in MCTS
- Create Self play class
- Create Arena class

DEEP MIND OPEN ACCESS PAPER 

https://kstatic.googleusercontent.com/files/2f51b2a749a284c2e2dfa13911da965f4855092a179469aedd15fbe4efe8f8cbf9c515ef83ac03a6515fa990e6f85fd827dcd477845e806f23a17845072dc7bd

RAY ALPHA ZERO IMPLEMENTATION 

https://github.com/ray-project/ray/tree/master/rllib/contrib/alpha_zero

DUPLICATED MCTS IMPLEMENTATION

https://github.com/suragnair/alpha-zero-general/blob/master/MCTS.py

DISTRIBUTED IMPLEMENTATION

https://github.com/mokemokechicken/reversi-alpha-zero/blob/master/src/reversi_zero/lib/ggf.py

CHESS MOVES

https://www.ichess.net/blog/chess-pieces-moves/

BOARD REPRESENTATIONS

https://medium.com/datadriveninvestor/reconstructing-chess-positions-f195fd5944e

ALPHA ZERO EXPLANATION

https://nikcheerla.github.io/deeplearningschool/2018/01/01/AlphaZero-Explained/

TRANSFORMER NETWORK IMPLEMENTATION

https://pytorch.org/tutorials/beginner/transformer_tutorial.html

In [ ]:
# !pip install --user python-chess gym colorama tqdm PyQt5 ray tensorboard

In [2]:
import chess, gym, pickle, random, torch, math
import chess.svg as svg
import numpy as np
import pandas as pd
from datetime import datetime
from copy import copy, deepcopy
import matplotlib.pyplot as plt
from torch.optim import Adam
import torch.nn as nn
from torch.nn import functional as F
from torch.distributions import Categorical
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms
# plt.style.use('ggplot')
from scipy.ndimage.interpolation import shift
from copy import deepcopy
from colorama import init, Fore, Back, Style
from collections import Counter, deque
from tqdm import tqdm, tnrange, notebook
from random import shuffle
import tkinter as tk
from tkinter import simpledialog
from PyQt5.QtSvg import QSvgWidget
from PyQt5.QtWidgets import QApplication, QWidget

from utilities import *
# import constants 
from constants import *
from models.vae import CNN_AE, CNN_VAE, train_ae, train_vae, Conv
from models.mdn import MDN_RNN, loss_function, clip_grad_norm_
from models.controller import Controller

# init(convert = True)
# REFERENCES
# Tensorflow implementation for Chess
# https://github.com/saurabhk7/chess-alpha-zero

# Pytorch implementation for Connect4
# https://github.com/plkmo/AlphaZero_Connect4/tree/master/src

In [ ]:
class Chess_Environment(gym.Env):
    """Chess Environment"""
    def __init__(self):
        self.board = chess.Board()
        self.white_pieces = ['P', 'N', 'B', 'R', 'Q', 'K']
        self.black_pieces = [piece.lower() for piece in self.white_pieces]
        self.x_coords, self.y_coords = np.meshgrid(list(range(0, 8)), list(range(0, 8)))
        self.x_coords = self.x_coords / 7
        self.y_coords = self.y_coords / 7
        self.state_size = self.observe()[0].shape
        self.init_action_decoder()
        self.whites_turn = True

    def reset(self):
        """Resets the environment"""
        self.board = chess.Board()
        
    def terminal_test(self):
        """Checks if the game is over"""
        return self.board.is_game_over(claim_draw = True)
    
    def result(self):
        """Gives the end game result"""

        result = self.board.result(claim_draw = True)
        if result == '1-0':
            return 1
        elif result == '0-1':
            return -1
        elif result == '1/2-1/2':
            return 0.1
        elif result == '*':
            return 0
        else:
            raise Exception('Invalid Result', result)
        
    def legal_actions(self):
        """Provides a list of legal actions in current state"""
        legal_actions = [str(legal_action) for legal_action in list(self.board.legal_moves)]
        return legal_actions
    
    def encode(self):
        """Encodes game state into a string"""
        board_ = self.board.piece_map()
            
        encoded = {
            'board' : board_,
            'turn' : self.board.turn,
            'legal_actions' : self.legal_actions()
        }
        
        return pickle.dumps(encoded)
    
    def decode(self, encoded):
        """Decodes string into game state and sets board and turn"""
        decoded = pickle.loads(encoded)
        self.board.set_piece_map(decoded['board'])
        if decoded['turn'] == False:
            self.board = self.board.mirror()
        self.board.turn = decoded['turn']

    def observe(self):
        """Create observation from the game state"""

        board_ = copy(self.board)
            
        board_ = np.ndarray.flatten(np.array(board_.__str__().split())).reshape(8, 8)
        
        black_pawns = np.isin(copy(board_), ['p']).astype(int)
        black_knights = np.isin(copy(board_), ['n']).astype(int)
        black_rooks = np.isin(copy(board_), ['r']).astype(int)
        black_bishops = np.isin(copy(board_), ['b']).astype(int)
        black_queen = np.isin(copy(board_), ['q']).astype(int)
        black_king = np.isin(copy(board_), ['k']).astype(int)
         
        white_pawns = np.isin(copy(board_), ['P']).astype(int)
        white_knights = np.isin(copy(board_), ['N']).astype(int)
        white_rooks = np.isin(copy(board_), ['R']).astype(int)
        white_bishops = np.isin(copy(board_), ['B']).astype(int)
        white_queen = np.isin(copy(board_), ['Q']).astype(int)
        white_king = np.isin(copy(board_), ['K']).astype(int)
        
        state = np.array([
            white_pawns,
            white_knights,
            white_rooks,
            white_bishops,
            white_queen,
            white_king,
            black_pawns,
            black_knights,
            black_rooks,
            black_bishops,
            black_queen,
            black_king
        ])
        
        return state, self.legal_actions()
    
    def step(self, action):
        """Perform a step in the environment"""
        try:
            self.board.push_uci(action)
        except ValueError:
            print(f"INVALID ACTION {action}.\nSELECTING A RANDOM ACTION FROM LEGAL ACTIONS\n{self.legal_actions()}")
            actions = random.choices(self.legal_actions())
            self.board.push_uci(actions[0])
            
        self.board = self.board.mirror()
        if self.whites_turn:
            self.whites_turn = False
        else:
            self.whites_turn = True
        return self.observe()
    
    def move_board(self, move):
        """Moves the board positions as per the move"""

        char_to_int = {'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6, 'G': 7, 'H': 8} 

        int_to_char = {v: k for k, v in char_to_int.items()}

        encoded_board = [(char_to_int[pos[0]], int(pos[1])) for pos in np.ndarray.flatten(BOARD)]

        new_board = [tuple(map(sum, zip((char_to_int[pos[0]], int(pos[1])), move))) for pos in np.ndarray.flatten(BOARD)]
        moves = []
        for pos, new_pos in zip(np.ndarray.flatten(BOARD), new_board):
            try:
                if new_pos[1] > 8:
                    raise Exception()
                if move[2] is None:
                    move_ = f'{pos}{int_to_char[new_pos[0]]}{new_pos[1]}'
                else:
                    move_ = f'{pos}{int_to_char[new_pos[0]]}{new_pos[1]}{move[2]}'
            
                if '-' in move_:
                    raise Exception()
            
                if '0' in move_:
                    raise Exception()
                
            except Exception:
                move_ = 'XXXX'
            moves.append(move_)   
        return np.array(moves).reshape(8, 8)
    
    def init_action_decoder(self):
        """Initialize the decoder to decode the actions"""
        decoder = []
        for key in MOVES.keys():
            decoder_ = self.move_board(MOVES[key])
            decoder.append(decoder_)
#             print(f'Move {key}\n', new_board, '\n')
    
        self.decoder = np.array(decoder)
        self.action_size = self.decoder.shape
        
    def select_action(self, logits):
        """Decodes the output from the NN to legal actions""" 
        decoder_ = np.ndarray.flatten(self.decoder)
        logits_ = np.ndarray.flatten(logits)

        move_logits = [(decoder_[idx].lower(), logits_[idx]) for idx in range(len(logits_))]
        move_logits = dict(move_logits)
        
        legal_move_logits = {legal_action: move_logits[legal_action] for legal_action in self.legal_actions()}
        probabilities = list(legal_move_logits.values()) / sum(list(legal_move_logits.values()))

        action = random.choices(list(legal_move_logits.keys()), weights = probabilities, k = 1)[0]
        return action
    
    def render(self):
        """Render chess board"""
        chess.svg.board(board = self.board)  

env = Chess_Environment()
env.board

In [ ]:
class Agent:
    
    def __init__(self, decoder, action_size, vae, mdn, controller, test_mode = False, batch_size = 64, lr = 1e-4, legal_multiplier = 2, save_freq = 2):
        self.batch_size = batch_size
        self.action_size = action_size
        self.legal_multiplier = legal_multiplier # To increase the value of legal actions and MCTS insight as the target policy
        self.decoder = decoder
        self.save_freq = save_freq
        if vae is not None:
            self.vae = CNN_AE(vae, None, 'Latest')
            self.vae.eval()
        if mdn is not None:
            self.mdn = MDN_RNN(mdn, None, 'Latest')
            params = {
                'hidden_size' : self.mdn.hidden_size,
                'z_size' : self.vae.z_size,
                'action_size' : action_size,
                'expansion_size': 1200
            }
            self.mdn.eval()
            if test_mode:
                self.controller = Controller('Test', params, 'Latest')
            else:
                self.controller = Controller('Test', params, False)
                self.controller.save_model(0)
        
            self.optimizer = Adam(self.controller.parameters(), lr = lr)
        
        self.char_to_int = {'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8} 
        self.char_to_int_promo = {'r': 1, 'b': 2, 'q': 3, 'k': 4, 'n': 5, '': 6} 
        
        moves_df = pd.DataFrame(list(zip(self.char_to_int.values(), self.char_to_int.keys())), columns = ['Ids', 'Labels'])
        promo_df = pd.DataFrame(list(zip(self.char_to_int_promo.values(), self.char_to_int_promo.keys())), columns = ['Ids', 'Labels'])
        self.move_actions_ohe = pd.get_dummies(moves_df['Ids'])
        self.promo_actions_ohe = pd.get_dummies(promo_df['Ids'])
        
        self.criterion_pi = nn.NLLLoss()
        self.criterion_v = nn.MSELoss()
        self.trained_times = 0
    
    def select_action(self, logits, legal_actions):
        """Decodes the output from the NN to legal actions""" 
        decoder_ = np.ndarray.flatten(self.decoder)
        logits_ = np.ndarray.flatten(logits)

        move_logits = [(decoder_[idx].lower(), logits_[idx]) for idx in range(len(logits_))]
        move_logits = dict(move_logits)
        
        legal_move_logits = {legal_action: move_logits[legal_action] for legal_action in legal_actions}
        probabilities = list(legal_move_logits.values()) / sum(list(legal_move_logits.values()))
        
        legal_actions = list(legal_move_logits.keys())
        action = random.choices(list(legal_move_logits.keys()), weights = probabilities, k = 1)[0]
        
        action_probabilities = {legal_actions[idx] : probabilities[idx] for idx in range(len(legal_actions))}
        return action, action_probabilities
    
    def one_hot_encode_action(self, action):
        """One hot encodes the action"""
        if len(action) == 4:
            encoded_action = [
                self.move_actions_ohe.loc[
                self.char_to_int[action[0]] - 1].values, 
                self.move_actions_ohe.loc[int(action[1]) - 1].values, 
                self.move_actions_ohe.loc[self.char_to_int[action[2]] - 1].values, 
                self.move_actions_ohe.loc[int(action[3]) - 1].values,
                self.promo_actions_ohe.loc[self.char_to_int_promo[''] - 1].values
            ]
        else:
            encoded_action = [
                self.move_actions_ohe.loc[
                self.char_to_int[action[0]] - 1].values, 
                self.move_actions_ohe.loc[int(action[1]) - 1].values, 
                self.move_actions_ohe.loc[self.char_to_int[action[2]] - 1].values, 
                self.move_actions_ohe.loc[int(action[3]) - 1].values,
                self.promo_actions_ohe.loc[self.char_to_int_promo[action[4]] - 1].values
            ]
        ohe_action = np.concatenate(encoded_action)
        return ohe_action
    
    def reset(self, batch_size):
        """Resets the MDNs hidden state"""
        self.hidden = self.mdn.init_hidden(batch_size)
        self.hidden = (self.hidden[0].detach().to('cpu'), self.hidden[1].detach().to('cpu'))
        
    def act(self, state, legal_actions):
        """Gets an action from the agent"""
        state = torch.tensor(state).float().unsqueeze(0)
        z = self.vae.encode(state)
    
        zh = torch.cat((z.to('cpu'), self.hidden[0].squeeze(0).to('cpu')), dim = 1)
        if zh.shape[0] > 1:
            raise Exception('Batch size > 1 not handled')
            
        logits, values = self.controller(zh)
        logits, value = logits.detach().squeeze(0).cpu().numpy(), values.detach().squeeze(0).cpu().numpy()[0]
        action, action_probabilities = self.select_action(logits, legal_actions)
        
        return action_probabilities, value, zh
    
    def encode_zh(self, state):
        """Encodes the state with the hidden state"""
        state = torch.tensor(state).float().unsqueeze(0)
        z = self.vae.encode(state)
        zh = torch.cat((z.to('cpu'), self.hidden[0].squeeze(0).to('cpu')), dim = 1)
        return zh
    
    def train(self, experiences):
        """Trains the controller from the MCTS experiences"""
        batches = [experiences[i : i + self.batch_size] for i in range(0, len(experiences), self.batch_size)]
        
        for batch in deepcopy(batches):
            zh, target_action_probs, returns = zip(*batch)
            returns = torch.tensor(returns).float()
            zh = torch.tensor(np.concatenate(zh, axis = 0)).float()
            self.optimizer.zero_grad()
            logits, values = self.controller(zh)
            target_policies = self.target_policies(logits, target_action_probs)
            
            policy_losses = []
            value_losses = []
            
            for idx in range(logits.shape[0]):
                advantage = returns[idx] - values[idx]
                # converts logits to action probabilities
#                 print(values[idx].squeeze(0).shape, returns[idx].shape)
#                 print(values[idx].squeeze(0), returns[idx])
                policy_losses.append(-F.softmax(logits[idx], dim = 0) * advantage)
                value_losses.append(F.smooth_l1_loss(values[idx].squeeze(0), returns[idx]))
            
            # sum up all the values of policy_losses and value_losses
            self.controller.train_steps += 1
            policy_loss = torch.stack(policy_losses).mean()
            value_loss = torch.stack(value_losses).mean()
            writer.add_scalar('policy_loss', policy_loss, self.controller.train_steps)
            writer.add_scalar('value_loss', value_loss, self.controller.train_steps)
            loss = policy_loss + value_loss
            
            print(loss)
            loss.backward()
            self.optimizer.step()
        if self.trained_times % self.save_freq == 0:
            print("SAVING")
            self.controller.save_model(self.trained_times)
        self.trained_times += 1
        
    def load_hidden(self, hidden):
        """Loads the hidden state to the MDN RNN"""
        self.hidden = deepcopy(hidden)
        
    def get_hidden(self):
        """Gets the hidden state from the MDN RNN"""
        return deepcopy(self.hidden)
    
    def random_controller(self): 
        """Updates the controller from a randomly selected save file"""
        self.controller.load_model('Random')
        
    def update_hidden(self, state, action):
        """Updates the hidden state of the MDN-RNN"""
        state = torch.tensor(state).float().unsqueeze(0)
        mu, logvar = self.vae.encode(state)
        z = self.vae.reparameterize(mu, logvar).squeeze(-1)
        ohe_action = self.one_hot_encode_action(action)
        za = torch.cat((z.to('cpu'), torch.tensor(ohe_action).float().to('cpu').unsqueeze(0)), dim = 1)
        _, self.hidden = self.mdn(za.unsqueeze(1), self.hidden)
        self.hidden = (self.hidden[0].detach().to('cpu'), self.hidden[1].detach().to('cpu'))
        
                
    def target_policies(self, logits, legal_move_logits):
        """Takes the logits from the current policy and layers the insights from MCTS"""
        new_boards__ = [[x.lower() for x in list(np.ndarray.flatten(self.decoder))] for _ in range(logits.shape[0])]
        target_policies = []
        for idx in range(logits.shape[0]):
            new_boards___ = new_boards__[idx]
            logits__ = logits[idx].view(-1)
            legal_move_logits__ = {k.lower(): v * self.legal_multiplier for k, v in legal_move_logits[idx].items()}
            for move, legal_move_logit in legal_move_logits__.items():
                index = new_boards___.index(move)
                logits__[index] += legal_move_logit
            target_policies.append(logits__)
        target_policies = torch.cat(target_policies, dim = 0).reshape(logits.shape[0], 76, 8, 8).float()
        return target_policies
    
agent = Agent(env.decoder, env.action_size, 'Test', 'Test', 'Test', test_mode = False)
agent.reset(1)

#### Encode State

In [ ]:
# agent = Agent(env.decoder, env.action_size, 'Test', 'Test', 'Test', test_mode = False)

# agent.reset(1)

# class MCTS():
#     """Monte Carlo Tree Search Algorithm geared for Neural Networks"""

#     def __init__(self, env, agent, mcts_simulations = 100, max_depth = 100, delta = 0.5):
#         self.env = env
#         self.agent = agent
#         self.cpuct = 0.2    # WARNING BULLSHIT NUMBER!
#         self.delta = delta  # value to prevent crash if no edges are visited
#         self.mcts_simulations = mcts_simulations
#         self.max_depth = max_depth
        
#         self.Qsa = {}       # stores Q values for s, a (as defined in the paper)
#         self.Nsa = {}       # stores # times edge s, a was visited
#         self.Ns = {}        # stores # times board s was visited
#         self.Ps = {}        # stores initial policy (returned by neural net)

#         self.Es = {}        # stores victory result (1, 0, -1) ended for board s
#         self.Vs = {}        # stores legal actions for board s
#         self.zh = {}        # stores preserved state (s) that is fed into the controller
#         self.hidden = {}    # stores the hidden state of the MDN-RNN of every (s,a) pair

#     def action_probabilities(self, encoded_state, temp = 1):
#         """
#         This function performs numMCTSSims simulations of MCTS starting from
#         canonicalBoard.
#         Returns:
#             probs: a policy vector where the probability of the ith action is
#                    proportional to Nsa[(s,a)]**(1./temp)
#         """
#         for runs in range(self.mcts_simulations):
# #             print(f"{Fore.BLUE}MCTS SIMULATION {runs + 1}{Style.RESET_ALL}")
#             self.search(encoded_state, 0)
        
#         s = encoded_state
#         self.env.decode(encoded_state)
#         legal_actions = self.env.legal_actions()
#         counts = [self.Nsa[(s,a)] if (s,a) in self.Nsa else 0 for a in legal_actions]
#         # counts array represent the number of time each action edge from your current state was traversed

#         if temp == 0: # temprature is 0 representing taking the best action possible (greedy)
#             bestA = np.argmax(counts) # bestA: best action number : argmax Returns the indices of the maximum values
#             probs = [0] * len(counts)
#             probs[bestA] = 1
#             action_probs = {legal_actions[idx] : probs[idx] for idx in range(len(legal_actions))}
#             return action_probs # returns the definite move(s) with same greedy reward, out of which one move HAS to be played
        
# #         print(f"{Fore.BLUE}COUNTS {sum(counts)} {counts}{Style.RESET_ALL}")
        
#         # Handles frequent draw situation when MCTS fails to explore when the game is over resulting 0 counts causing div 0 error
#         if sum(counts) == 0:
#             counts = [1 for _ in counts]
#         counts = [x ** (1. / temp) + 0.5 for x in counts]
#         probs = [x / float(sum(counts)) for x in counts]
#         action_probs = {legal_actions[idx] : probs[idx] for idx in range(len(legal_actions))}
#         # If the game isn't over return the state zh or return None at the end
# #         if self.env.result() != 0:
# #             return action_probs, None
#         return action_probs, self.zh[s]  
#         #returns the probablity of different moves that CAN be played resulting in uniform distribution


#     def search(self, encoded_state, depth):
#         """
#         This function performs one iteration of MCTS. It is recursively called
#         till a leaf node is found. The action chosen at each node is one that
#         has the maximum upper confidence bound as in the paper.
#         Once a leaf node is found, the neural network is called to return an
#         initial policy P and a value v for the state. This value is propogated
#         up the search path. In case the leaf node is a terminal state, the
#         outcome is propogated up the search path. The values of Ns, Nsa, Qsa are
#         updated.
#         NOTE: the return values are the negative of the value of the current
#         state. This is done since v is in [-1,1] and if v is the value of a
#         state for the current player, then its value is -v for the other player.
#         Returns:
#             v: the negative of the value of the current canonicalBoard
#         """

#         s = deepcopy(encoded_state)
        
#         # loads the appropriate hidden state for MDN-RNN else saves it
#         if s not in self.hidden:
#             self.hidden[s] = self.agent.get_hidden()
#         else:
#             self.agent.load_hidden(self.hidden[s])
        
#         self.env.decode(encoded_state)
# #         print(f"{Fore.GREEN}SEARCH DEPTH {depth}{Style.RESET_ALL}")
        
#         # Check if its an terminal state, -1 Opponent Won, 0 Game not Over, 1 Player Won
#         if s not in self.Es:
#             self.Es[s] = self.env.result()
#         if self.Es[s] != 0 or self.env.terminal_test():
#             state, _ = self.env.observe()
#             self.zh[s] = self.agent.encode_zh(state) 
# #             print("SIMULATION OVER!", self.env.terminal_test(), -self.Es[s], self.env.board.result())
#             return -self.Es[s]
        
#         state, legal_actions = self.env.observe()
# #         legal_actions = self.env.legal_actions()

#         if s not in self.Ps: #if the current state 's' is not explored/expanded before n=0 by MCTS then create a new node and rollout
#             self.Ps[s], v, self.zh[s] = self.agent.act(state, legal_actions)
# #             print("self.Ps[s] at Depth", depth, "\n", self.Ps[s])
#             valids = legal_actions

#             self.Vs[s] = valids 
#             self.Ns[s] = 0
# #             print("VALUE", -v)
#             return -v
        
#         valids = self.Vs[s] #as already visited the valid moves array 'Vs' is already initialized
#         cur_best = -float('inf')
#         best_act = -1
        
#         self.env.decode(encoded_state)
#         legal_actions = self.env.legal_actions()
#         # pick the action with the highest upper confidence bound
        
#         agent_actions = list(self.Ps[s].keys())
        
#         # Handles the occasional legal action that isn't an actual legal action
#         # Pawn side cut even though there is no enemy piece in respective position
# #         print("XXXXX PRE LEGAL ACTIONS\n", legal_actions)
#         if not set(agent_actions) == set(legal_actions):
#             legal_actions = agent_actions
# #         print("XXXXX POST LEGAL ACTIONS\n", legal_actions)
#         for a in legal_actions:
#             if (s,a) in self.Qsa:
#                 u = self.Qsa[(s,a)] + self.cpuct * self.Ps[s][a] * math.sqrt(self.Ns[s]) / (1 + self.Nsa[(s,a)])
# #                 print("In Qsa")
#             else:
#                 self.Ps[s][a] 
#                 u = self.cpuct * self.Ps[s][a] * math.sqrt(self.Ns[s])     # Q = 0 ? : node exists but not explored as added and initilized during nnet phase
# #                 print("Not In Qsa", self.Ps[s][a], self.Ns[s], math.sqrt(self.Ns[s]))
# #             print("U", u)
#             if math.isnan(u):
#                 u = 0
# #                 print("Override U", u)
#             if u > cur_best:
#                 cur_best = u
#                 best_act = a
# #         print(f"{Fore.GREEN}BEST ACTION {best_act}{Style.RESET_ALL}")
#         a = best_act
#         self.agent.update_hidden(state, a)
        
        
#         _, _ = self.env.step(a)
#         encoded_next_state = self.env.encode()

#         v = self.search(encoded_next_state, depth + 1) # RECURSION until leaf node or terminal node is found
        
#         self.env.decode(encoded_state)
        
#         if (s,a) in self.Qsa:
#             self.Qsa[(s,a)] = (self.Nsa[(s,a)] * self.Qsa[(s,a)] + v) / (self.Nsa[(s,a)] + 1) #update the Q Value
#             self.Nsa[(s,a)] += 1 # increment number of visits to this node in MCTS
#         else:
#             self.Qsa[(s,a)] = v # initialize the new node
#             self.Nsa[(s,a)] = 1

#         self.Ns[s] += 1
#         return -v
    
#     def reset(self):
#         """Resets the tracked information"""
#         self.Qsa = {}       
#         self.Nsa = {}       
#         self.Ns = {}        
#         self.Ps = {}       
#         self.Es = {}
#         self.Vs = {}  
#         self.zh = {}
#         self.agent.reset(1)
#         self.env.reset()

# for _ in range(5):
#     env.reset()
#     mcts_white = MCTS(deepcopy(env), deepcopy(agent), mcts_simulations = 20)
#     mcts_black = MCTS(deepcopy(env), deepcopy(agent), mcts_simulations = 20)
#     game_move = 0
#     while not env.terminal_test():
#     #     print(f"{Fore.RED}GAME MOVE {game_move}{Style.RESET_ALL}")
#         encoded_state = env.encode()
#         if env.whites_turn: 
#             action_probs, zh = mcts_white.action_probabilities(encoded_state)
#         else:
#             action_probs, zh = mcts_black.action_probabilities(encoded_state)
        
        
# #         print(game_move, env.whites_turn, env.terminal_test(), type(zh))
# #         if zh == None:
# #             print(encoded_state)
#         action = random.choices(list(action_probs.keys()), weights = action_probs.values(), k = 1)[0]
# #         print(f"{Fore.CYAN}ACTUAL MOVE\n {action_probs}{Style.RESET_ALL}")
#         _, _ = env.step(action)
    
#         game_move += 1    
#     print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! GAME COMPLETED !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")


In [ ]:
# state, legal_actions = env.observe()

# state_planes = [
#     'White Pawns',
#     'White Knights',
#     'White Rooks',
#     'White Bishops',
#     'White Queen',
#     'White King',
#     'Black Pawns',
#     'Black Knights',
#     'Black Rooks',
#     'Black Bishops',
#     'Black Queen',
#     'Black King'
# ]

# for inx, state_ in enumerate(state):
#     fig = plt.figure()
#     fig.suptitle(state_planes[inx], fontsize = 20)
#     xticks = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']
#     yticks = list(range(1, 9))
#     yticks.reverse()
#     plt.xticks(list(range(0, 8)), xticks)
#     plt.yticks(list(range(0, 8)), yticks)
#     plt.imshow(state_)


#### Encode Moves

In [ ]:
# queen_directions = ['N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW']
# queen_distance = list(range(1, 8))
# queen_moves = [f'{direction}{distance}' for direction in queen_directions for distance in queen_distance]

# knight_moves = ['2N1E', '1N2E', '1S2E', '2S1E', '2S1W', '1S2W', '1N2W', '2N1W']
# underpromotion_moves = ['DOUBLEM', 'NECUT', 'NWCUT']
# moves = queen_moves + knight_moves + underpromotion_moves

# print(f'{len(queen_moves)} QUEEN MOVES {queen_moves[: 15]}\n{len(knight_moves)} KNIGHT MOVES {knight_moves}\n{len(underpromotion_moves)} UNDERPROMOTION MOVES {underpromotion_moves}')
# print(f'\nTOTAL MOVES {len(moves)}')
# print(moves)

In [ ]:
# def move_board(move):
#     """Moves the board positions as per the move"""
#     print(move)
#     board = np.array([
#         ['A8', 'B8', 'C8', 'D8', 'E8', 'F8', 'G8', 'H8'],
#         ['A7', 'B7', 'C7', 'D7', 'E7', 'F7', 'G7', 'H7'],
#         ['A6', 'B6', 'C6', 'D6', 'E6', 'F6', 'G6', 'H6'],
#         ['A5', 'B5', 'C5', 'D5', 'E5', 'F5', 'G5', 'H5'],
#         ['A4', 'B4', 'C4', 'D4', 'E4', 'F4', 'G4', 'H4'],
#         ['A3', 'B3', 'C3', 'D3', 'E3', 'F3', 'G3', 'H3'],
#         ['A2', 'B2', 'C2', 'D2', 'E2', 'F2', 'G2', 'H2'],
#         ['A1', 'B1', 'C1', 'D1', 'E1', 'F1', 'G1', 'H1']
#     ])

#     char_to_int = {'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6, 'G': 7, 'H': 8} 

#     int_to_char = {v: k for k, v in char_to_int.items()}

#     encoded_board = [(char_to_int[pos[0]], int(pos[1])) for pos in np.ndarray.flatten(board)]

#     new_board = [tuple(map(sum, zip((char_to_int[pos[0]], int(pos[1])), move))) for pos in np.ndarray.flatten(board)]
 
#     moves = []
#     for pos, new_pos in zip(np.ndarray.flatten(board), new_board):
#         try:
# #             print(pos, new_pos)
#             if new_pos[1] > 8:
#                 raise Exception()
            
#             if move[2] is None:
#                 move_ = f'{pos}{int_to_char[new_pos[0]]}{new_pos[1]}'
#             else:
#                 move_ = f'{pos}{int_to_char[new_pos[0]]}{new_pos[1]}{move[2]}'
            
#             if '-' in move_:
#                 raise Exception()
            
#             if '0' in move_:
#                 raise Exception()
                
#         except Exception:
#             move_ = 'XXXX'
#         moves.append(move_)
#     return np.array(moves).reshape(8, 8)

# moves = {
#     # Queen Moves
#     'Q N1': (0, 1, None),
#     'Q N2': (0, 2, None),
#     'Q N3': (0, 3, None),
#     'Q N4': (0, 4, None),
#     'Q N5': (0, 5, None),
#     'Q N6': (0, 6, None),
#     'Q N7': (0, 7, None),
#     'Q NE1': (1, 1, None),
#     'Q NE2': (2, 2, None),
#     'Q NE3': (3, 3, None),
#     'Q NE4': (4, 4, None),
#     'Q NE5': (5, 5, None),
#     'Q NE6': (6, 6, None),
#     'Q NE7': (7, 7, None),
#     'Q E1': (1, 0, None),
#     'Q E2': (2, 0, None),
#     'Q E3': (3, 0, None),
#     'Q E4': (4, 0, None),
#     'Q E5': (5, 0, None),
#     'Q E6': (6, 0, None),
#     'Q E7': (7, 0, None),
#     'Q SE1': (1, -1, None),
#     'Q SE2': (2, -2, None),
#     'Q SE3': (3, -3, None),
#     'Q SE4': (4, -4, None),
#     'Q SE5': (5, -5, None),
#     'Q SE6': (6, -6, None),
#     'Q SE7': (7, -7, None),
#     'Q S1': (0, -1, None),
#     'Q S2': (0, -2, None),
#     'Q S3': (0, -3, None),
#     'Q S4': (0, -4, None),
#     'Q S5': (0, -5, None),
#     'Q S6': (0, -6, None),
#     'Q S7': (0, -7, None),
#     'Q SW1': (-1, -1, None),
#     'Q SW2': (-2, -2, None),
#     'Q SW3': (-3, -3, None),
#     'Q SW4': (-4, -4, None),
#     'Q SW5': (-5, -5, None),
#     'Q SW6': (-6, -6, None),
#     'Q SW7': (-7, -7, None),
#     'Q W1': (-1, 0, None),
#     'Q W2': (-2, 0, None),
#     'Q W3': (-3, 0, None),
#     'Q W4': (-4, 0, None),
#     'Q W5': (-5, 0, None),
#     'Q W6': (-6, 0, None),
#     'Q W7': (-7, 0, None),
#     'Q NW1': (-1, 1, None),
#     'Q NW2': (-2, 2, None),
#     'Q NW3': (-3, 3, None),
#     'Q NW4': (-4, 4, None),
#     'Q NW5': (-5, 5, None),
#     'Q NW6': (-6, 6, None),
#     'Q NW7': (-7, 7, None),
#     # Knight Moves
#     '2N1E' : (1, 2, None),
#     '1N2E' : (2, 1, None),
#     '1S2E' : (2, -1, None),
#     '2S1E' : (1, -2, None),
#     '2S1W' : (-1, -2, None),
#     '1S2W' : (-2, -1, None),
#     '1N2W' : (-2, 1, None),
#     '2N1W' : (-1, 2, None),
#     # Promtions
#     'PQ N1' : (0, 1, 'q'),
#     'PR N1' : (0, 1, 'r'),
#     'PB N1' : (0, 1, 'b'),
#     'PN N1' : (0, 1, 'n'),
#     'PQ NE1' : (1, 1, 'q'),
#     'PR NE1' : (1, 1, 'r'),
#     'PB NE1' : (1, 1, 'b'),
#     'PN NE1' : (1, 1, 'n'),
#     'PQ NW1' : (-1, 1, 'q'),
#     'PR NW1' : (-1, 1, 'r'),
#     'PB NW1' : (-1, 1, 'b'),
#     'PN NW1' : (-1, 1, 'n')
# }

# new_boards = []
# for key in moves.keys():
#     print(f'Move {key}')
#     new_board = move_board(moves[key])
#     new_boards.append(new_board)
#     print(new_board, '\n')
    
# new_boards = np.array(new_boards)
# new_boards.shape

#### Decode Logits

In [ ]:
# # logits = env.action
# logits = np.random.rand(len(moves), 8, 8)

# print(env.action_size)
# new_boards_ = np.ndarray.flatten(new_boards)
# logits_ = np.ndarray.flatten(logits)

# move_logits = [(new_boards_[idx].lower(), logits_[idx]) for idx in range(len(logits_))]
# move_logits = dict(move_logits)

# legal_actions = ['g1h3','g1f3','b1c3','b1a3','h2h3','g2g3','f2f3','e2e3','d2d3','c2c3','b2b3','a2a3','h2h4','g2g4','f2f4','e2e4','d2d4','c2c4','b2b4','a2a4']

# legal_move_logits = {legal_action: move_logits[legal_action] for legal_action in legal_actions}
# legal_move_logits

#### Encode Logits FOR CONTROLLER TRAINING LOOP

In [ ]:
# LEGAL_MULTIPIER = 20
# BATCH_SIZE = 64

# legal_move_logits_ = [legal_move_logits for _ in range(BATCH_SIZE)] 
# logits = np.random.rand(BATCH_SIZE, len(moves), 8, 8)
# logits_ = np.swapaxes(logits.reshape(-1, BATCH_SIZE), 0, 1)
# new_boards__ = [[x.lower() for x in list(new_boards_)] for _ in range(BATCH_SIZE)]
# target_policies = []
# for idx in range(BATCH_SIZE):
#     new_boards___ = new_boards__[idx]
#     logits__ = logits_[idx]
#     legal_move_logits__ = {k.lower(): v * LEGAL_MULTIPIER for k, v in legal_move_logits_[idx].items()}
#     for move, legal_move_logit in legal_move_logits__.items():
#         index = new_boards___.index(move)
#         logits__[index] += legal_move_logit
#     target_policies.append(logits__)
# target_policies = np.array(target_policies).reshape(BATCH_SIZE, len(moves), 8, 8)

#### Create Training Data for VAE and figure out action size

In [ ]:
# def create_vae_training_data(games):
#     """Creates training date for CNN-VAE"""
#     actions = {}
#     states = {}
#     for game in range(1, games + 1):
#         env.reset()
#         state, legal_actions = env.observe()
#         encoded_state = env.encode()
#         states[encoded_state] = state
#         while not env.terminal_test():
#             action = random.choice(legal_actions)            
#             state, legal_actions = env.step(action)
#             encoded_state = env.encode()
#             states[encoded_state] = state
# #             actions[encoded_state] = agent.one_hot_encode_action(action)
#         print(f'Game {game} | Unique States {len(states.keys())} Unique Actions {len(actions)}', end = '\r')
    
#     validate_path('data/vae')
#     with open('data/vae/states.pkl', 'wb') as file:
#         pickle.dump(states, file)
#     with open('data/vae/actions.pkl', 'wb') as file:
#         pickle.dump(actions, file)
    
# create_vae_training_data(50)

#### Train VAE

In [ ]:
# def train_vae_(name, starting_channels, filename, z_size, epochs, batch_size):
#     """Trains the CNN-VAE"""
#     params = {
#         'z_size' : z_size,
#         'batch_size' : batch_size,
#         'learning_rate' : 1e-4,
#         'kl_tolerance' : 0.5,
#         'batch_norm' : True,
#         'starting_channels' : starting_channels
#     }

#     vae = CNN_AE(name, params, False)
#     with open(f'data/vae/{filename}',  'rb') as pickle_file:
#         data = list(pickle.load(pickle_file).values())
#         data = [np.array(data_) for data_ in data]
    
#     train_ae(vae, data, epochs, 100, 2)

    
# train_vae_('Test', env.state_size[0], 'states.pkl', 300, 1000, 200)
# # train_vae_('Test Actions', env.action_size[0], 'actions.pkl', 600, 100, 200)

#### Testing VAE

In [ ]:
# def test_vae(name, starting_channels, filename, z_size):
#     """Visualize Test of the CNN-VAE"""
    
#     with open(f'data/vae/{filename}',  'rb') as pickle_file:
#         data = list(pickle.load(pickle_file).values())
#         data = [np.array(data_) for data_ in data]
    
#     vae = CNN_AE(name, None, 'Latest')
#     vae = vae.to(vae.device)

#     test_loader = DataLoader(
#         torch.tensor(data).float(),
#         batch_size = vae.batch_size, 
#         shuffle = True
#     )
#     count = 0
    
#     for batch_idx, inputs in enumerate(test_loader):
#         if inputs.shape[0] != vae.batch_size:
#             continue
#         inputs = inputs.to(vae.device)
#         outputs = vae(inputs)
#         axes = []
#         for idx in range(inputs.shape[0]):
#             sample_input = transforms.ToPILImage(mode = 'RGB')(inputs[idx].squeeze(0).detach().cpu())
#             sample_output = transforms.ToPILImage(mode = 'RGB')(outputs[idx].squeeze(0).detach().cpu()) 
#             fig, ax = plt.subplots(1,2)
#             ax[0].imshow(sample_input)
#             ax[1].imshow(sample_output)
#             plt.show()
# #         columns = 5
# #         for i, images_ in enumerate(images):

# #             plt.subplot(len(images) / columns + 1, columns, i + 1)
# #             plt.imshow(image)
            
    
    
# test_vae('Test', env.state_size[0], 'test_states.pkl', 600)

#### Create Training Data for MDN

In [ ]:
# def create_mdn_training_data(name, games):
#     """Create training data for MDN-RNN"""
#     vae = CNN_AE(name, None, 'Latest')
#     vae.eval()
#     data = pd.DataFrame(columns = ['Game ID', 'Sequence #', 'State ID'])
#     rollouts = []
#     transitions = 0
    
#     for game in range(1, games + 1):
#         white_states = []
#         white_actions = []
#         black_states = []
#         black_actions = []
#         env.reset()
    
#         state, legal_actions = env.observe()
#         while not env.terminal_test():
#             action = random.choice(legal_actions)
#             state, legal_actions = env.step(action)
#             transitions += 1
#             if env.whites_turn:
#                 white_states.append(state)
#                 white_actions.append(agent.one_hot_encode_action(action))
#             else:
#                 black_states.append(state)
#                 black_actions.append(agent.one_hot_encode_action(action))             
            
#         white_zs = vae.encode(torch.tensor(white_states).float())
#         white_actions = torch.tensor(white_actions).float().squeeze(-1)
#         black_zs = vae.encode(torch.tensor(black_states).float())
#         black_actions = torch.tensor(black_actions).float().squeeze(-1)        
        
# #         white_zas = torch.cat([white_zs, white_actions], dim = 1)
# #         black_zas = torch.cat([black_zs, black_actions], dim = 1)
#         rollouts.append((white_zs.detach().numpy(), white_actions.detach().numpy()))
#         rollouts.append((black_zs.detach().numpy(), black_actions.detach().numpy()))
        
#         print(f'Game {game} Transitions {transitions}', end = '\r')
    
#     validate_path('data/mdn')
#     with open('data/mdn/rollouts.pkl', 'wb') as file:
#         pickle.dump(rollouts, file)
    
    
# create_mdn_training_data('Test', 600)

In [ ]:
# def training_mdn(vae_name, name):
#     torch.cuda.empty_cache() 
#     """Trains the MDN-RNN"""
#     epochs = 499
#     batch_size = 5
    
#     params = {
#         'hidden_size' : 150,
#         'gaussian_size' : 3,
#         'stacked_layers': 1,
#         'grad_clip' : 0.5,
#         'learning_rate' : 1e-4
#     }
    
#     with open(f'data/mdn/rollouts.pkl', 'rb') as pickle_file:
#         rollouts = pickle.load(pickle_file)
    
#     params['z_size'] = rollouts[0][0].shape[1]
#     params['action_size'] = rollouts[0][1].shape[1]
#     params['batch_size'] = batch_size
#     vae = CNN_AE(vae_name, None, 'Latest')
#     vae.eval()
#     vae = vae.to(vae.device)
#     mdn = MDN_RNN(name, params, False)
    
#     mdn.train()
#     mdn = mdn.to(mdn.device)
#     optimizer = Adam(mdn.parameters(), lr = mdn.learning_rate)
        
#     inputs = []
#     prev_states = []
#     targets = []
#     seq_lengths = []
    
# #     print("ROLLOUTS", len(rollouts))
#     for rollout in rollouts:
#         zs, actions = rollout
#         zas = np.concatenate((zs, actions), axis = 1)
#         seq_lengths.append(zas[ : -1].shape[0])
#         inputs.append(zas[ : -1])
#         targets.append(zs[1 : ])
#         prev_states.append(zs[ : -1])
        
#     idxs = range(len(rollouts))
    
#     # Sorts order from longest to shortest sequence
#     idxs = [x for _, x in sorted(zip(seq_lengths, idxs), reverse = True)]
#     seq_lengths = [seq_lengths[x] for x in idxs]
#     inputs = [np.array(inputs[x]) for x in idxs]
#     targets = [np.array(targets[x]) for x in idxs]
#     prev_states = [np.array(prev_states[x]) for x in idxs]
    
#     max_seq_len = max(seq_lengths)
#     rollout_size = len(seq_lengths)
# #     print("ROLLOUT SIZE", rollout_size, "MAX SEQ", max_seq_len)
    
#     padded_X = np.zeros((rollout_size, max_seq_len, inputs[0].shape[-1]))
#     padded_Y = np.zeros((rollout_size, max_seq_len, targets[0].shape[-1]))
#     padded_Z = np.zeros((rollout_size, max_seq_len, prev_states[0].shape[-1]))
# #     https://towardsdatascience.com/taming-lstms-variable-sized-mini-batches-and-why-pytorch-is-good-for-your-health-61d35642972e
    
#     for i, x_len in enumerate(seq_lengths):
#         padded_X[i, 0 : x_len] = inputs[i]
#         padded_Y[i, 0 : x_len] = targets[i]
#         padded_Z[i, 0 : x_len] = prev_states[i]
        
#     inputs = torch.tensor(padded_X).float().squeeze(1).to(mdn.device)
#     targets = torch.tensor(padded_Y).float().squeeze(1).to(mdn.device)
#     prev_states = torch.tensor(padded_Z).float().squeeze(1).to(mdn.device)
    
# #     print("X1", inputs.shape, targets.shape)
#     if inputs.shape[0] % batch_size != 0:
#         inputs = inputs[inputs.shape[0] // batch_size, :, :]
# #     print("X2", inputs.shape, targets.shape)    
#     inputs = inputs.reshape(-1, batch_size, max_seq_len, mdn.z_size + mdn.action_size)
#     targets = targets.reshape(-1, batch_size, max_seq_len, mdn.z_size)
#     prev_states = prev_states.reshape(-1, batch_size, max_seq_len, mdn.z_size)
# #     print("X3", inputs.shape, targets.shape)  
    
#     seq_lengths = list(torch.tensor(seq_lengths).reshape(-1, batch_size).numpy())
    
#     for epoch in range(epochs + 1):
#         for batch in range(inputs.shape[0]):
#             inputs_ = inputs[batch]
#             targets_ = targets[batch]
#             prev_states_ = prev_states[batch]
#             seq_lengths_ = list(seq_lengths[batch])
            
#             targets_ = torch.nn.utils.rnn.pack_padded_sequence(targets_, seq_lengths_, batch_first = True)
#             targets_, _ = torch.nn.utils.rnn.pad_packed_sequence(targets_, batch_first = True)

#             # Set initial hidden and cell states
#             hidden = mdn.init_hidden(batch_size)

#             # Forward pass
#             hidden = (hidden[0].detach(), hidden[1].detach())
#             (pi, mu, sigma), hidden = mdn(inputs_, hidden, seq_lengths_)
#             loss = loss_function(targets_, pi, mu, sigma)
            
#             outputs_ = torch.stack([torch.normal(mu, sigma)[:, :, i, :] for i in range(mdn.gaussian_size)]).float()
            
#             random_sample = random.randrange(targets_.shape[1])
# #             print("TARGETS", targets_.shape, "OUTPUTS", outputs_.shape, "PREV STATES", prev_states_.shape, "RANDOM", random_sample)
            
#             target_d = targets_[0][random_sample].unsqueeze(0).to(vae.device)
#             output_d = outputs_[: , 0, random_sample].to(vae.device)
#             prev_state_d = prev_states_[0][random_sample].unsqueeze(0).to(vae.device)
# #             print("TARGET D", target_d.shape, "OUTPUT D", output_d.shape, "PREV STATE D", prev_state_d.shape, "RANDOM", random_sample)
            
#             output_d = [output_d[x].unsqueeze(0) for x in range(output_d.shape[0])]
# #             print("FLAT OUTPUT D", len(output_d), output_d[0].shape)
#             x = torch.stack([prev_state_d, target_d] + output_d).squeeze(1).float()
# #             print("DECODE X", x.shape)
#             compare_x = vae.decode(x)
            
#             sample_prev_state = compare_x[0].squeeze(0).detach().cpu()
#             sample_target = compare_x[1].squeeze(0).detach().cpu()
#             sample_outputs = [compare_x[idx].squeeze(0).detach().cpu() for idx in range(1, compare_x.shape[0])] 
            
#             sample_target = transforms.ToPILImage(mode = 'RGB')(sample_target)
#             sample_prev_state = transforms.ToPILImage(mode = 'RGB')(sample_prev_state)
#             sample_outputs = [transforms.ToPILImage(mode = 'RGB')(sample_output) for sample_output in sample_outputs]
            
#             fig, ax = plt.subplots(1, compare_x.shape[0], figsize = (15, 15))
#             ax[0].imshow(sample_prev_state)
#             ax[1].imshow(sample_target)
#             for idx, sample_output in enumerate(sample_outputs):
#                 ax[1 + idx].imshow(sample_output)
#             plt.show()
            
# #             print(compare_x.shape)
#             # Backward and optimize
#             mdn.zero_grad()
#             loss.backward()
#             clip_grad_norm_(mdn.parameters(), mdn.grad_clip)
#             optimizer.step()
#         print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch, epochs, loss.item()))
            
#         mdn.save_model(epoch)
# training_mdn('Test', 'Test')

# ALPHA GO TIPS 


## MCTS
Each simulation proceeds by
selecting in each state s a move a with low visit count, high move probability and high value
(averaged over the leaf states of simulations that selected a from s) according to the current
neural network fθ. The search returns a vector π representing a probability distribution over
moves, either proportionally or greedily with respect to the visit counts at the root state.
The parameters θ of the deep neural network in AlphaZero are trained by self-play reinforcement learning, starting from randomly initialised parameters θ. Games are played by selecting
moves for both players by MCTS, at ∼ πt
. At the end of the game, the terminal position sT is
scored according to the rules of the game to compute the game outcome z: −1 for a loss, 0 for
a draw, and +1 for a win. The neural network parameters θ are updated so as to minimise the
error between the predicted outcome vt and the game outcome z, and to maximise the similarity
of the policy vector pt
to the search probabilities πt
. Specifically, the parameters θ are adjusted
by gradient descent on a loss function l that sums over mean-squared error and cross-entropy
losses respectively

(p, v) = fθ(s)                    

l = (z − v) 2 − π > log p + c||θ||2


In [ ]:
agent = Agent(env.decoder, env.action_size, 'Test', 'Test', 'Test', test_mode = False)

agent.reset(1)

In [ ]:
env.decode(b'\x80\x03}q\x00(X\x05\x00\x00\x00boardq\x01}q\x02(K?cchess\nPiece\nq\x03)\x81q\x04}q\x05(X\n\x00\x00\x00piece_typeq\x06K\x04X\x05\x00\x00\x00colorq\x07\x89ubK9h\x03)\x81q\x08}q\t(h\x06K\x02h\x07\x89ubK6h\x03)\x81q\n}q\x0b(h\x06K\x06h\x07\x89ubK0h\x03)\x81q\x0c}q\r(h\x06K\x01h\x07\x89ubK.h\x03)\x81q\x0e}q\x0f(h\x06K\x01h\x07\x88ubK)h\x03)\x81q\x10}q\x11(h\x06K\x01h\x07\x89ubK\x1fh\x03)\x81q\x12}q\x13(h\x06K\x02h\x07\x88ubK\x1eh\x03)\x81q\x14}q\x15(h\x06K\x03h\x07\x89ubK\x1dh\x03)\x81q\x16}q\x17(h\x06K\x06h\x07\x88ubK\x1ch\x03)\x81q\x18}q\x19(h\x06K\x01h\x07\x89ubK\x1ah\x03)\x81q\x1a}q\x1b(h\x06K\x01h\x07\x88ubK\x18h\x03)\x81q\x1c}q\x1d(h\x06K\x02h\x07\x89ubK\x17h\x03)\x81q\x1e}q\x1f(h\x06K\x01h\x07\x88ubK\x01h\x03)\x81q }q!(h\x06K\x02h\x07\x88ubuX\x04\x00\x00\x00turnq"\x88X\r\x00\x00\x00legal_actionsq#]q$(X\x04\x00\x00\x00h4f5q%X\x04\x00\x00\x00h4f3q&X\x04\x00\x00\x00h4g2q\'X\x04\x00\x00\x00f4g5q(X\x04\x00\x00\x00f4e5q)X\x04\x00\x00\x00f4g4q*X\x04\x00\x00\x00f4e4q+X\x04\x00\x00\x00f4g3q,X\x04\x00\x00\x00f4e3q-X\x04\x00\x00\x00b1c3q.X\x04\x00\x00\x00b1a3q/X\x04\x00\x00\x00b1d2q0X\x04\x00\x00\x00h3g4q1X\x04\x00\x00\x00c4c5q2eu.')
print(env.legal_actions())
env.board


In [ ]:
class MCTS():
    """Monte Carlo Tree Search Algorithm geared for Neural Networks"""

    def __init__(self, env, agent, mcts_simulations = 100, max_depth = 100, delta = 0.5):
        self.env = env
        self.agent = agent
        self.cpuct = 0.2    # WARNING BULLSHIT NUMBER!
        self.delta = delta  # value to prevent crash if no edges are visited
        self.mcts_simulations = mcts_simulations
        self.max_depth = max_depth
        
        self.Qsa = {}       # stores Q values for s, a (as defined in the paper)
        self.Nsa = {}       # stores # times edge s, a was visited
        self.Ns = {}        # stores # times board s was visited
        self.Ps = {}        # stores initial policy (returned by neural net)

        self.Es = {}        # stores victory result (1, 0, -1) ended for board s
        self.Vs = {}        # stores legal actions for board s
        self.zh = {}        # stores preserved state that is fed into the controller

    def action_probabilities(self, encoded_state, temp = 1):
        """
        This function performs numMCTSSims simulations of MCTS starting from
        canonicalBoard.
        Returns:
            probs: a policy vector where the probability of the ith action is
                   proportional to Nsa[(s,a)]**(1./temp)
        """
        for runs in range(self.mcts_simulations):
#             print(f"{Fore.BLUE}MCTS SIMULATION {runs + 1}{Style.RESET_ALL}")
            self.search(encoded_state, 0)
        
        s = encoded_state
        self.env.decode(encoded_state)
        legal_actions = self.env.legal_actions()
        counts = [self.Nsa[(s,a)] if (s,a) in self.Nsa else 0 for a in legal_actions]
        # counts array represent the number of time each action edge from your current state was traversed

        if temp == 0: # temprature is 0 representing taking the best action possible (greedy)
            bestA = np.argmax(counts) # bestA: best action number : argmax Returns the indices of the maximum values
            probs = [0] * len(counts)
            probs[bestA] = 1
            action_probs = {legal_actions[idx] : probs[idx] for idx in range(len(legal_actions))}
            return action_probs # returns the definite move(s) with same greedy reward, out of which one move HAS to be played
        
#         print(f"{Fore.BLUE}COUNTS {sum(counts)} {counts}{Style.RESET_ALL}")
        
        # Handles frequent draw situation when MCTS fails to explore when the game is over resulting 0 counts causing div 0 error
        if sum(counts) == 0:
            counts = [1 for _ in counts]
        counts = [x ** (1. / temp) + 0.5 for x in counts]
        probs = [x / float(sum(counts)) for x in counts]
        action_probs = {legal_actions[idx] : probs[idx] for idx in range(len(legal_actions))}
        
        # If the game isn't over return the state zh or return None at the end
#         if self.env.result() != 0:
#             return action_probs, None
        return action_probs, self.zh[s]  
        #returns the probablity of different moves that CAN be played resulting in uniform distribution


    def search(self, encoded_state, depth):
        """
        This function performs one iteration of MCTS. It is recursively called
        till a leaf node is found. The action chosen at each node is one that
        has the maximum upper confidence bound as in the paper.
        Once a leaf node is found, the neural network is called to return an
        initial policy P and a value v for the state. This value is propogated
        up the search path. In case the leaf node is a terminal state, the
        outcome is propogated up the search path. The values of Ns, Nsa, Qsa are
        updated.
        NOTE: the return values are the negative of the value of the current
        state. This is done since v is in [-1,1] and if v is the value of a
        state for the current player, then its value is -v for the other player.
        Returns:
            v: the negative of the value of the current canonicalBoard
        """

        s = deepcopy(encoded_state)
        self.env.decode(encoded_state)
#         print(f"{Fore.GREEN}SEARCH DEPTH {depth}{Style.RESET_ALL}")
        
        # Check if its an terminal state, -1 Opponent Won, 0 Game not Over, 1 Player Won
        if s not in self.Es:
            self.Es[s] = self.env.result()
        if self.Es[s] != 0 or self.env.terminal_test():
            state, _ = self.env.observe()
            self.zh[s] = self.agent.encode_zh(state)
            
#             print("SIMULATION OVER!", self.env.terminal_test(), -self.Es[s], self.env.board.result())
            return -self.Es[s]
        
        state, legal_actions = self.env.observe()
#         legal_actions = self.env.legal_actions()

        if s not in self.Ps: #if the current state 's' is not explored/expanded before n=0 by MCTS then create a new node and rollout
            self.Ps[s], v, self.zh[s] = self.agent.act(state, legal_actions)
#             print("self.Ps[s] at Depth", depth, "\n", self.Ps[s])
            valids = legal_actions

            self.Vs[s] = valids 
            self.Ns[s] = 0
#             print("VALUE", -v)
            return -v
        
        valids = self.Vs[s] #as already visited the valid moves array 'Vs' is already initialized
        cur_best = -float('inf')
        best_act = -1
        
        self.env.decode(encoded_state)
        legal_actions = self.env.legal_actions()
        # pick the action with the highest upper confidence bound
        
        agent_actions = list(self.Ps[s].keys())
        
        # Handles the occasional legal action that isn't an actual legal action
        # Pawn side cut even though there is no enemy piece in respective position
#         print("XXXXX PRE LEGAL ACTIONS\n", legal_actions)
        if not set(agent_actions) == set(legal_actions):
            legal_actions = agent_actions
#         print("XXXXX POST LEGAL ACTIONS\n", legal_actions)
        for a in legal_actions:
            if (s,a) in self.Qsa:
                u = self.Qsa[(s,a)] + self.cpuct * self.Ps[s][a] * math.sqrt(self.Ns[s]) / (1 + self.Nsa[(s,a)])
#                 print("In Qsa")
            else:
                self.Ps[s][a] 
                u = self.cpuct * self.Ps[s][a] * math.sqrt(self.Ns[s])     # Q = 0 ? : node exists but not explored as added and initilized during nnet phase
#                 print("Not In Qsa", self.Ps[s][a], self.Ns[s], math.sqrt(self.Ns[s]))
#             print("U", u)
            if math.isnan(u):
                u = 0
#                 print("Override U", u)
            if u > cur_best:
                cur_best = u
                best_act = a
#         print(f"{Fore.GREEN}BEST ACTION {best_act}{Style.RESET_ALL}")
        a = best_act

        _, _ = self.env.step(a)
        encoded_next_state = self.env.encode()

        v = self.search(encoded_next_state, depth + 1) # RECURSION until leaf node or terminal node is found
        
        self.env.decode(encoded_state)
        
        if (s,a) in self.Qsa:
            self.Qsa[(s,a)] = (self.Nsa[(s,a)] * self.Qsa[(s,a)] + v) / (self.Nsa[(s,a)] + 1) #update the Q Value
            self.Nsa[(s,a)] += 1 # increment number of visits to this node in MCTS
        else:
            self.Qsa[(s,a)] = v # initialize the new node
            self.Nsa[(s,a)] = 1

        self.Ns[s] += 1
        return -v
    
    def reset(self):
        """Resets the tracked information"""
        self.Qsa = {}       
        self.Nsa = {}       
        self.Ns = {}        
        self.Ps = {}       
        self.Es = {}
        self.Vs = {}  
        self.zh = {}
        self.hidden = {}
        self.agent.reset(1)
        self.env.reset()

# for _ in range(5):
#     env.reset()
#     mcts_white = MCTS(deepcopy(env), deepcopy(agent), mcts_simulations = 20)
#     mcts_black = MCTS(deepcopy(env), deepcopy(agent), mcts_simulations = 20)
#     game_move = 0
#     while not env.terminal_test():
#     #     print(f"{Fore.RED}GAME MOVE {game_move}{Style.RESET_ALL}")
#         encoded_state = env.encode()
#         if env.whites_turn: 
#             action_probs, zh = mcts_white.action_probabilities(encoded_state)
#         else:
#             action_probs, zh = mcts_black.action_probabilities(encoded_state)
        
        
# #         print(game_move, env.whites_turn, env.terminal_test(), type(zh))
# #         if zh == None:
# #             print(encoded_state)
#         action = random.choices(list(action_probs.keys()), weights = action_probs.values(), k = 1)[0]
# #         print(f"{Fore.CYAN}ACTUAL MOVE\n {action_probs}{Style.RESET_ALL}")
#         _, _ = env.step(action)
    
#         game_move += 1    
#     print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! GAME COMPLETED !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")

### Training Loop

In [ ]:
# # COPY THIS
# # https://github.com/suragnair/alpha-zero-general/blob/master/Coach.py

# env.reset()
# mcts_white = MCTS(deepcopy(env), deepcopy(agent), mcts_simulations = 20)
# mcts_black = MCTS(deepcopy(env), deepcopy(agent), mcts_simulations = 20)
# game_move = 0

# while not env.terminal_test():
#     print(f"{Fore.RED}GAME MOVE {game_move}{Style.RESET_ALL}")
#     encoded_state = env.encode()
#     if env.whites_turn: 
#         action_probs = mcts_white.action_probabilities(encoded_state)
#     else:
#         action_probs = mcts_black.action_probabilities(encoded_state)
#     action = random.choices(list(action_probs.keys()), weights = action_probs.values(), k = 1)[0]
#     print(f"{Fore.CYAN}ACTUAL MOVE\n {action_probs}{Style.RESET_ALL}")
#     _, _ = env.step(action)
    
#     game_move += 1
        
# print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! GAME COMPLETED !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")

In [ ]:
# env.decode(b'\x80\x03}q\x00(X\x05\x00\x00\x00boardq\x01}q\x02(K<cchess\nPiece\nq\x03)\x81q\x04}q\x05(X\n\x00\x00\x00piece_typeq\x06K\x02X\x05\x00\x00\x00colorq\x07\x88ubK9h\x03)\x81q\x08}q\t(h\x06K\x06h\x07\x89ubK8h\x03)\x81q\n}q\x0b(h\x06K\x04h\x07\x89ubK5h\x03)\x81q\x0c}q\r(h\x06K\x01h\x07\x89ubK1h\x03)\x81q\x0e}q\x0f(h\x06K\x01h\x07\x89ubK0h\x03)\x81q\x10}q\x11(h\x06K\x01h\x07\x89ubK)h\x03)\x81q\x12}q\x13(h\x06K\x01h\x07\x88ubK&h\x03)\x81q\x14}q\x15(h\x06K\x01h\x07\x89ubK\x1dh\x03)\x81q\x16}q\x17(h\x06K\x01h\x07\x89ubK\x1ch\x03)\x81q\x18}q\x19(h\x06K\x01h\x07\x88ubK\x1bh\x03)\x81q\x1a}q\x1b(h\x06K\x01h\x07\x89ubK\x1ah\x03)\x81q\x1c}q\x1d(h\x06K\x01h\x07\x88ubK\x19h\x03)\x81q\x1e}q\x1f(h\x06K\x02h\x07\x89ubK\x14h\x03)\x81q }q!(h\x06K\x03h\x07\x89ubK\x0ch\x03)\x81q"}q#(h\x06K\x03h\x07\x88ubK\x08h\x03)\x81q$}q%(h\x06K\x01h\x07\x88ubK\x06h\x03)\x81q&}q\'(h\x06K\x04h\x07\x89ubK\x05h\x03)\x81q(}q)(h\x06K\x06h\x07\x88ubK\x01h\x03)\x81q*}q+(h\x06K\x02h\x07\x88ubK\x00h\x03)\x81q,}q-(h\x06K\x04h\x07\x88ubuX\x04\x00\x00\x00turnq.\x88X\r\x00\x00\x00legal_actionsq/]q0u.')
# env.board

In [ ]:
class Coach():
    """
    This class executes the self-play + learning. It uses the functions defined
    in Game and NeuralNet. args are specified in main.py.
    """
    
    def __init__(self, env, agent, params):
        self.iterations = params['iterations']
        self.episodes = params['episodes']
        self.queue_length = params['queue_length']
        self.env = deepcopy(env)
        self.agent = agent
        self.mcts_white = MCTS(deepcopy(env), deepcopy(agent), mcts_simulations = params['simulations'])
        self.mcts_black = MCTS(deepcopy(env), deepcopy(agent), mcts_simulations = params['simulations'])
        self.trainExamplesHistory = []  # history of examples from args.numItersForTrainExamplesHistory latest iterations
        self.games = 0
        self.writer = SummaryWriter(f'runs/{self.controller.name}/{datetime.now()}/{self.games}')

    def executeEpisode(self, writer):
        """
        This function executes one episode of self-play, starting with player 1.
        As the game is played, each turn is added as a training example to
        trainExamples. The game is played till the game ends. After the game
        ends, the outcome of the game is used to assign values to each example
        in trainExamples.
        It uses a temp=1 if episodeStep < tempThreshold, and thereafter
        uses temp=0.
        Returns:
            trainExamples: a list of examples of the form (canonicalBoard, currPlayer, pi,v)
                           pi is the MCTS informed policy vector, v is +1 if
                           the player eventually won the game, else -1.
        """
        experiences = []
        self.env.reset()
        # RESET HIDDEN STATES!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
        game_move = 0
        while not self.env.terminal_test():
            encoded_state = self.env.encode()
            if self.env.whites_turn: 
                action_probs, zh = self.mcts_white.action_probabilities(encoded_state, writer)
            else:
                action_probs, zh = self.mcts_black.action_probabilities(encoded_state)
            if zh is not None:
                print("ZH NOT NONE")
                experiences.append([zh.detach().cpu().numpy(), self.env.whites_turn, action_probs, None])

            else:
                print("ZH NONE)
            action = random.choices(list(action_probs.keys()), weights = action_probs.values(), k = 1)[0]
            _, _ = self.env.step(action)
            
            if not self.env.whites_turn: 
                      
    
            game_move += 1
        
        reward = self.env.result()
        print(f"Game {self.games} Score {reward}")
        experiences_ = []
        DISCOUNT_GAMMA = 0.95
        
        for idx, experience in enumerate(experiences):
            reward_ = reward
            # calculates the discounted return from the reward.. 
            # / 2 becauses it contains experiences of both black and white
            return_ = reward_ * (DISCOUNT_GAMMA ** (len(experiences) - idx) / 2)
            if experience[1] != self.env.whites_turn and (reward_ == 1 or reward_ == -1):
                return_ *= -1
            experiences_.append((experience[0], experience[2], return_))
        return experiences_
    
    
    def learn(self):
        """
        Performs numIters iterations with numEps episodes of self-play in each
        iteration. After every iteration, it retrains neural network with
        examples in trainExamples (which has a maximum length of maxlenofQueue).
        It then pits the new neural network against the old one and accepts it
        only if it wins >= updateThreshold fraction of games.
        """
        # deletes historical experiences so AI to enable online learning
        self.trainExamplesHistory = []
        self.writer.games = self.games
        
        for i in range(1, self.iterations + 1):
            iterationTrainExamples = deque([], maxlen = self.queue_length)
            for _ in tnrange(self.episodes, desc = "Self Play"):
                iterationTrainExamples += self.executeEpisode(deepcopy(self.writer))
                self.games += 1
                self.writer.games += 1

            # save the iteration examples to the history 
            self.trainExamplesHistory.append(iterationTrainExamples)

        # shuffle examples before training
        trainExamples = []
        for e in self.trainExamplesHistory:
            trainExamples.extend(e)
        shuffle(trainExamples)
        print(len(trainExamples))
        
        self.agent.train(trainExamples, deepcopy(self.writer))
        self.mcts_white.agent = deepcopy(self.agent)
        self.mcts_black.agent = deepcopy(self.agent)
        
        self.mcts_white.reset()  # reset search tree as agents brain is now different
        self.mcts_black.reset()
        

#         # training new network, keeping a copy of the old one
#         self.nnet.save_checkpoint(folder=self.args.checkpoint, filename='temp.pth.tar')
#         self.pnet.load_checkpoint(folder=self.args.checkpoint, filename='temp.pth.tar')
#         pmcts = MCTS(self.game, self.pnet, self.args)

#         
#         nmcts = MCTS(self.game, self.nnet, self.args)

#         log.info('PITTING AGAINST PREVIOUS VERSION')
#         arena = Arena(lambda x: np.argmax(pmcts.getActionProb(x, temp=0)),
#                           lambda x: np.argmax(nmcts.getActionProb(x, temp=0)), self.game)
#         pwins, nwins, draws = arena.playGames(self.args.arenaCompare)

#         log.info('NEW/PREV WINS : %d / %d ; DRAWS : %d' % (nwins, pwins, draws))
#         if pwins + nwins == 0 or float(nwins) / (pwins + nwins) < self.args.updateThreshold:
#             log.info('REJECTING NEW MODEL')
#             self.nnet.load_checkpoint(folder=self.args.checkpoint, filename='temp.pth.tar')
#         else:
#             log.info('ACCEPTING NEW MODEL')
#             self.nnet.save_checkpoint(folder=self.args.checkpoint, filename=self.getCheckpointFile(i))
#             self.nnet.save_checkpoint(folder=self.args.checkpoint, filename='best.pth.tar')
        

params = {
    'iterations': 1,
    'episodes': 50,
    'queue_length' : 500,
    'simulations' : 4
}
# coach = Coach(env, agent, params)
# for _ in range(500):
#     coach.learn()

In [ ]:
class MainWindow(QWidget):
    def __init__(self):
        super().__init__()

        self.setGeometry(100, 100, 1100, 1100)

        self.widgetSvg = QSvgWidget(parent=self)
        self.widgetSvg.setGeometry(10, 10, 1080, 1080)


# board = chess.Board()
# app = QApplication([])
# window = MainWindow()
# window.show()
# app.exec()

In [ ]:
import matplotlib.pyplot as plotter
agent = Agent(env.decoder, env.action_size, 'Test', 'Test', 'Test', test_mode = True)
agent.reset(1)

class Arena:
    """An Arena where you can have two players fight"""
    
    def __init__(self, env, agent):
        self.env = env        
        self.agent = agent
        self.root = tk.Tk()
        self.root.withdraw()
        self.app = QApplication([])
        self.window = MainWindow()
    
    def vs_human(self):
        """Fight Against the AI"""
        self.agent = Agent(self.env.decoder, self.env.action_size, 'Test', 'Test', 'Test', test_mode = False)
        self.mcts_black = MCTS(deepcopy(self.env), deepcopy(self.agent), mcts_simulations = 20)
        
        self.env.reset()
        self.mcts_black.reset()
        game_move = 0
        while not self.env.terminal_test():
            if not self.env.whites_turn:
                encoded_state = self.env.encode()
                action_probs, zh = self.mcts_black.action_probabilities(encoded_state)
                
                figureObject, axesObject = plotter.subplots()
                # Draw the pie chart
                axesObject.pie(list(action_probs.values()), labels = list(action_probs.keys()), autopct = '%1.2f', startangle = 90)
                # Aspect ratio - equal means pie is a circl
                axesObject.axis('equal')
                plotter.show()
                
                
                action = random.choices(list(action_probs.keys()), weights = action_probs.values(), k = 1)[0]
            else:
                legal_actions = self.env.legal_actions() 
                self.render()
                action = simpledialog.askstring(title = "Move", prompt = f"What's your next move?\n{legal_actions}")
                while action not in legal_actions:
                    action = simpledialog.askstring(title = "Move", prompt = f"Invalid Move. What's your next move?\n{legal_actions}")
                    if action is None:
                        break
                if action is None:
                    break
            
            _, _ = self.env.step(action)
            game_move += 1   
        print(env.result())
        
    def render(self):
        """Renders the chess board"""
        chessboardSvg = chess.svg.board(self.env.board).encode("UTF-8")
        self.window.widgetSvg.load(chessboardSvg)
        self.window.show()
        self.app.exec()
        
        
arena = Arena(env, agent)
arena.vs_human()